<a href="https://colab.research.google.com/github/A-Wpro/ML_Predict_Movie_revenue/blob/main/movieMLpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data set
https://www.kaggle.com/tmdb/tmdb-movie-metadata

WIP

TODO LIST :
-time stamp to unix
-PCA
-LABEL ENCODER ? instead of dummuies ?
-scoring and co 
-find best model

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
import datetime


# Movie feat :  adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count

In [ ]:
movies_metadata=pd.read_csv('/content/drive/My Drive/movies_metadata.csv', sep=',')


movies_metadataWithoutUselessFeat = movies_metadata
movies_metadataWithoutUselessFeat = movies_metadataWithoutUselessFeat.drop(movies_metadataWithoutUselessFeat.columns[[1,3, 4,5,6,8,9,11,12,13,17,18, 19,20,21]], axis=1)


movies_metadataWithoutNaN = movies_metadataWithoutUselessFeat.dropna()
print("\nShape de la frame :\n", movies_metadataWithoutNaN.shape) # on  perd 1500 lignes au lieu de 40k en enlevant 3 feats

#TODO : les demmuies, date to unix seconds

movies_metadataWithoutNaN.release_date = movies_metadataWithoutNaN.release_date.values 

print(movies_metadataWithoutNaN.release_date.values.astype('float'))


FileNotFoundError: ignored

In [ ]:
#les dummies; on regarde deja commetn ca se passe
movies_metadataWithoutNaN=pd.get_dummies(movies_metadataWithoutNaN, columns=['adult','original_language'])#,''])



In [ ]:
X = movies_metadataWithoutNaN.drop(5) #13= revenue; if revenue = 0 then no money was win
y = movies_metadataWithoutNaN.iloc[:,5]
y = y.iloc[1:]
print('y :',y.shape)
print('x :',X.shape)
print(X.dtypes)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.9, random_state=42)

In [ ]:
knn = KNeighborsClassifier(7)
knn.fit(X_train,y_train)

# Print the score
print(knn.score(X_test, y_test))
